In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

# Load Citi Bike 28-day tabular data
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")


In [4]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,  # or df_ts or your actual DataFrame
    cutoff_date=datetime(2025, 1, 1, 0, 0, 0),  # within your date range
    target_column="target"  # actual target column name
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(1665, 674)
(1665,)
(600, 674)
(600,)


In [5]:
# Select only lag features (exclude pickup_hour, pickup_location_id)
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]


In [7]:
import lightgbm as lgb

model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13373
[LightGBM] [Info] Number of data points in the train set: 1665, number of used features: 672
[LightGBM] [Info] Start training from score 0.592192


LGBMRegressor()

In [8]:
from sklearn.metrics import mean_absolute_error

predictions = model.predict(X_test_only_numeric)
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


0.9140


In [9]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.9140


In [10]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv

load_dotenv()
mlflow = set_mlflow_tracking()

log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressor-Citibike", "mean_absolute_error", score=test_mae)


INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/05/10 05:04:54 INFO mlflow.tracking.fluent: Experiment with name 'LGBMRegressor-Citibike' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: LGBMRegressor-Citibike
INFO:src.experiment_utils:Logged mean_absolute_error: 0.9139865237421014
c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www

2025/05/10 05:05:07 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'LGBMRegressor'.
2025/05/10 05:05:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBMRegressor, version 1
Created version '1' of model 'LGBMRegressor'.
INFO:src.experiment_utils:Model logged with name: LGBMRegressor


🏃 View run debonair-flea-646 at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/4/runs/a400956eca30415794d7daa5aed181c6
🧪 View experiment at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/4
